<a href="https://colab.research.google.com/github/juanmg1984/dm2025a/blob/main/MAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps tqdm

import pandas as pd
import googlemaps
import time
import io
from google.colab import files
from tqdm.notebook import tqdm
from getpass import getpass

# ---------------------------------------------------------------
# 1. LÍMITE DE CONSULTAS API
# ---------------------------------------------------------------
LIMITE_API = 1000
contador_api = 0
limite_alcanzado = False
# ---------------------------------------------------------------

# ---------------------------------------------------------------
# 2. Pide la API Key de forma segura
# ---------------------------------------------------------------
API_KEY = getpass("Ingresa tu API Key de Google Maps (con 'Routes API' y 'Places API' habilitadas): ")
# ---------------------------------------------------------------

try:
    # Inicializa el cliente de Google Maps
    gmaps = googlemaps.Client(key=API_KEY)

    # ---------------------------------------------------------------
    # 3. Subir el archivo
    # ---------------------------------------------------------------
    print("Por favor, sube tu archivo 'Distancias - Hoja 1.csv'")
    uploaded = files.upload()

    # Obtiene el nombre del archivo subido
    file_name = list(uploaded.keys())[0]
    print(f"Archivo '{file_name}' cargado.")

    # ---------------------------------------------------------------
    # 4. Cargar el CSV en un DataFrame (con UTF-8)
    # ---------------------------------------------------------------
    try:
        # Forzamos UTF-8 para leer 'Ñ' y acentos correctamente
        df = pd.read_csv(io.BytesIO(uploaded[file_name]), encoding='utf-8')
    except UnicodeDecodeError:
        print("Error de UTF-8, intentando con 'latin-1'...")
        df = pd.read_csv(io.BytesIO(uploaded[file_name]), encoding='latin-1')

    print("DataFrame cargado exitosamente. Esta es la cabecera:")
    print(df.head())

    # --- Solución a las advertencias de Pandas ---
    # Convertimos todas las columnas de destino a tipo 'object' (texto)
    columnas_destino = df.columns[4:]
    df[columnas_destino] = df[columnas_destino].astype(object)

    # ---------------------------------------------------------------
    # 5. Procesar la Matriz
    # ---------------------------------------------------------------

    # Un caché para guardar resultados
    cache_distancias = {}

    print(f"Iniciando cálculo de distancias para {len(df)} orígenes y {len(columnas_destino)} destinos.")
    print(f"El script se detendrá automáticamente al alcanzar {LIMITE_API} consultas a la API.")
    print("Esto puede tardar MUCHO tiempo.")

    # Usamos tqdm para tener una barra de progreso
    for index in tqdm(df.index, desc="Procesando Orígenes"):

        if limite_alcanzado:
            break # Detiene el bucle de orígenes si se alcanzó el límite

        # Construye el string de Origen
        origen = f"{df.at[index, 'DestinatarioPoblacion']}, {df.at[index, 'DestinatarioRegion']}"

        for col_name in columnas_destino:

            # Construye el string de Destino
            destino = col_name

            # --- SOLO CALCULA SI LA CELDA ESTÁ VACÍA ---
            if pd.isna(df.at[index, col_name]) or str(df.at[index, col_name]).strip() == "":

                # Revisa si la ruta ya está en el caché
                cache_key = (origen, destino)
                if cache_key in cache_distancias:
                    dist_km = cache_distancias[cache_key]

                else:
                    # --- ¡NUEVO! Chequeo de Límite de API ---
                    if contador_api >= LIMITE_API:
                        print(f"\n--- LÍMITE DE {LIMITE_API} CONSULTAS ALCANZADO ---")
                        print("Deteniendo el script. El archivo se guardará con el progreso actual.")
                        limite_alcanzado = True
                        break # Detiene el bucle de destinos

                    # Incrementa el contador ANTES de hacer la llamada
                    contador_api += 1

                    # --- Si no está en caché y no hay límite, llama a la API ---
                    try:
                        # Imprime un log de vez en cuando para ver qué está haciendo
                        if contador_api % 50 == 0:
                            print(f"(Consulta API #{contador_api}) Calculando: {origen} -> {destino}")

                        directions_result = gmaps.directions(origen,
                                                             destino,
                                                             mode="driving")

                        if directions_result: # Revisa si la respuesta no está vacía
                            distance_meters = directions_result[0]['legs'][0]['distance']['value']
                            dist_km = distance_meters / 1000
                        else:
                            dist_km = "Error: ZERO_RESULTS"

                        cache_distancias[cache_key] = dist_km
                        time.sleep(0.05) # Pausa para no saturar la API

                    except Exception as e:
                        error_msg = str(e)
                        if "REQUEST_DENIED" in error_msg:
                            dist_km = "Error: REQUEST_DENIED"
                        elif "ZERO_RESULTS" in error_msg:
                            dist_km = "Error: ZERO_RESULTS"
                        else:
                            dist_km = f"Error: {error_msg.split('(')[0]}"

                        print(f"Error en ruta: {origen} -> {destino}. Error: {dist_km}")
                        cache_distancias[cache_key] = dist_km

                # Escribe el resultado (KM o Error) en el DataFrame
                df.at[index, col_name] = dist_km

    # ---------------------------------------------------------------
    # 6. Guardar y Descargar el Resultado (se ejecuta siempre)
    # ---------------------------------------------------------------
    output_filename = 'Distancias_Completadas.csv'
    # Guardamos en UTF-8-SIG para que Excel lea bien las 'Ñ' y acentos
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    if limite_alcanzado:
        print(f"\n--- PROCESO DETENIDO POR LÍMITE DE {LIMITE_API} CONSULTAS ---")
    else:
        print("\n--- ¡PROCESO COMPLETADO! ---")

    print(f"El archivo '{output_filename}' (con el progreso actual) está listo para descargar.")

    # Ofrece la descarga del archivo
    files.download(output_filename)

except Exception as e:
    print(f"\nHa ocurrido un error general: {e}")
    if 'API_KEY' not in locals() or not API_KEY:
        print("Error: La API Key no fue ingresada.")
    if 'uploaded' not in locals():
        print("Error: No se subió ningún archivo.")

Ingresa tu API Key de Google Maps (con 'Routes API' y 'Places API' habilitadas): ··········
Por favor, sube tu archivo 'Distancias - Hoja 1.csv'


Saving Distancias - Hoja 1.csv to Distancias - Hoja 1 (5).csv
Archivo 'Distancias - Hoja 1 (5).csv' cargado.
DataFrame cargado exitosamente. Esta es la cabecera:
    DC    DCDescripcion DestinatarioPoblacion DestinatarioRegion  \
0  101  CAPITAL FEDERAL       CAPITAL FEDERAL    Capital Federal   
1  102          ALBERTI               ALBERTI       Buenos Aires   
2  103  ALMIRANTE BROWN               ADROGUE       Buenos Aires   
3  103  ALMIRANTE BROWN               BURZACO       Buenos Aires   
4  103  ALMIRANTE BROWN              CLAYPOLE       Buenos Aires   

   7C8R+R7, Isidro Casanova, Provincia de Buenos Aires  \
0                                               24.0     
1                                              200.0     
2                                               23.0     
3                                               24.0     
4                                               34.0     

   3RF8+5F Plaza Huincul, Neuquén, Argentina  \
0                               

Procesando Orígenes:   0%|          | 0/1113 [00:00<?, ?it/s]

Error en ruta: CAPITAL FEDERAL, Capital Federal -> 0021. Error: Error: NOT_FOUND
Error en ruta: CAPITAL FEDERAL, Capital Federal -> 0023. Error: Error: NOT_FOUND
Error en ruta: CAPITAL FEDERAL, Capital Federal -> 0024. Error: Error: NOT_FOUND
Error en ruta: CAPITAL FEDERAL, Capital Federal -> 5207. Error: Error: NOT_FOUND
Error en ruta: ALBERTI, Buenos Aires -> 0021. Error: Error: NOT_FOUND
Error en ruta: ALBERTI, Buenos Aires -> 0023. Error: Error: NOT_FOUND
Error en ruta: ALBERTI, Buenos Aires -> 0024. Error: Error: NOT_FOUND
Error en ruta: ALBERTI, Buenos Aires -> 5207. Error: Error: NOT_FOUND
Error en ruta: ADROGUE, Buenos Aires -> 0021. Error: Error: NOT_FOUND
Error en ruta: ADROGUE, Buenos Aires -> 0023. Error: Error: NOT_FOUND
Error en ruta: ADROGUE, Buenos Aires -> 0024. Error: Error: NOT_FOUND
(Consulta API #50) Calculando: ADROGUE, Buenos Aires -> 0643
Error en ruta: ADROGUE, Buenos Aires -> 5207. Error: Error: NOT_FOUND
Error en ruta: BURZACO, Buenos Aires -> 0021. Error: Er

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# --- 1. Instalar bibliotecas ---
!pip install googlemaps tqdm

# --- 2. Importar todo ---
import pandas as pd
import googlemaps
import time
import io
from google.colab import files
from tqdm.notebook import tqdm
from google.colab import userdata  # Para leer los secretos

# ---------------------------------------------------------------
# 3. LÍMITE DE CONSULTAS API
# ---------------------------------------------------------------
LIMITE_API = 1000
contador_api = 0
limite_alcanzado = False
# ---------------------------------------------------------------

# ---------------------------------------------------------------
# 4. Lee la API Key desde los Secretos de Colab
# ---------------------------------------------------------------
try:
    # Busca un secreto guardado con el nombre 'maps'
    API_KEY = userdata.get('maps')
except userdata.SecretNotFoundError:
    print("Error: No se encontró el secreto 'maps'.")
    print("Por favor, haz clic en el ícono de la llave (🔑) a la izquierda y guarda tu API key con el nombre 'maps'.")
    raise # Detiene el script
except Exception as e:
    print(f"Error al leer el secreto: {e}")
    raise
# ---------------------------------------------------------------


try:
    # Inicializa el cliente de Google Maps
    gmaps = googlemaps.Client(key=API_KEY)
    print("API Key leída y cliente de Google Maps inicializado correctamente.")

    # ---------------------------------------------------------------
    # 5. Subir el archivo
    # ---------------------------------------------------------------
    print("Por favor, sube tu archivo 'Distancias - Hoja 1.csv'")
    uploaded = files.upload()

    # Obtiene el nombre del archivo subido
    file_name = list(uploaded.keys())[0]
    print(f"Archivo '{file_name}' cargado.")

    # ---------------------------------------------------------------
    # 6. Cargar el CSV en un DataFrame (con UTF-8)
    # ---------------------------------------------------------------
    try:
        df = pd.read_csv(io.BytesIO(uploaded[file_name]), encoding='utf-8')
    except UnicodeDecodeError:
        print("Error de UTF-8, intentando con 'latin-1'...")
        df = pd.read_csv(io.BytesIO(uploaded[file_name]), encoding='latin-1')

    print("DataFrame cargado exitosamente. Esta es la cabecera:")
    print(df.head())

    # --- Solución a las advertencias de Pandas ---
    columnas_destino = df.columns[4:]
    df[columnas_destino] = df[columnas_destino].astype(object)

    # ---------------------------------------------------------------
    # 7. Procesar la Matriz
    # ---------------------------------------------------------------

    # Un caché para guardar resultados
    cache_distancias = {}

    print(f"Iniciando cálculo de distancias para {len(df)} orígenes y {len(columnas_destino)} destinos.")
    print(f"El script se detendrá automáticamente al alcanzar {LIMITE_API} consultas a la API.")
    print("Esto puede tardar MUCHO tiempo.")

    # Usamos tqdm para tener una barra de progreso
    for index in tqdm(df.index, desc="Procesando Orígenes"):

        if limite_alcanzado:
            break # Detiene el bucle de orígenes si se alcanzó el límite

        # Construye el string de Origen
        origen = f"{df.at[index, 'DestinatarioPoblacion']}, {df.at[index, 'DestinatarioRegion']}"

        for col_name in columnas_destino:

            # Construye el string de Destino
            destino = col_name

            # --- SOLO CALCULA SI LA CELDA ESTÁ VACÍA ---
            if pd.isna(df.at[index, col_name]) or str(df.at[index, col_name]).strip() == "":

                # Revisa si la ruta ya está en el caché
                cache_key = (origen, destino)
                if cache_key in cache_distancias:
                    dist_km = cache_distancias[cache_key]

                else:
                    # --- ¡NUEVO! Chequeo de Límite de API ---
                    if contador_api >= LIMITE_API:
                        print(f"\n--- LÍMITE DE {LIMITE_API} CONSULTAS ALCANZADO ---")
                        print("Deteniendo el script. El archivo se guardará con el progreso actual.")
                        limite_alcanzado = True
                        break # Detiene el bucle de destinos

                    # Incrementa el contador ANTES de hacer la llamada
                    contador_api += 1

                    # --- Si no está en caché y no hay límite, llama a la API ---
                    try:
                        # Imprime un log de vez en cuando para ver qué está haciendo
                        if contador_api % 50 == 0:
                            print(f"(Consulta API #{contador_api}) Calculando: {origen} -> {destino}")

                        directions_result = gmaps.directions(origen,
                                                             destino,
                                                             mode="driving")

                        if directions_result: # Revisa si la respuesta no está vacía
                            distance_meters = directions_result[0]['legs'][0]['distance']['value']
                            dist_km = distance_meters / 1000
                        else:
                            dist_km = "Error: ZERO_RESULTS"

                        cache_distancias[cache_key] = dist_km
                        time.sleep(0.05) # Pausa para no saturar la API

                    except Exception as e:
                        error_msg = str(e)
                        if "REQUEST_DENIED" in error_msg:
                            dist_km = "Error: REQUEST_DENIED"
                        elif "ZERO_RESULTS" in error_msg:
                            dist_km = "Error: ZERO_RESULTS"
                        else:
                            dist_km = f"Error: {error_msg.split('(')[0]}"

                        print(f"Error en ruta: {origen} -> {destino}. Error: {dist_km}")
                        cache_distancias[cache_key] = dist_km

                # Escribe el resultado (KM o Error) en el DataFrame
                df.at[index, col_name] = dist_km

    # ---------------------------------------------------------------
    # 8. Guardar y Descargar el Resultado (se ejecuta siempre)
    # ---------------------------------------------------------------
    output_filename = 'Distancias_Completadas.csv'
    # Guardamos en UTF-8-SIG para que Excel lea bien las 'Ñ' y acentos
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    if limite_alcanzado:
        print(f"\n--- PROCESO DETENIDO POR LÍMITE DE {LIMITE_API} CONSULTAS ---")
    else:
        print("\n--- ¡PROCESO COMPLETADO! ---")

    print(f"El archivo '{output_filename}' (con el progreso actual) está listo para descargar.")

    # Ofrece la descarga del archivo
    files.download(output_filename)

except Exception as e:
    print(f"\nHa ocurrido un error general: {e}")
    if 'API_KEY' not in locals() or not API_KEY:
        print("Error: La API Key no fue ingresada o leída correctamente.")
    if 'uploaded' not in locals():
        print("Error: No se subió ningún archivo.")

API Key leída y cliente de Google Maps inicializado correctamente.
Por favor, sube tu archivo 'Distancias - Hoja 1.csv'


Saving Distancias - Hoja 1.csv to Distancias - Hoja 1 (4).csv
Archivo 'Distancias - Hoja 1 (4).csv' cargado.
DataFrame cargado exitosamente. Esta es la cabecera:
    DC    DCDescripcion DestinatarioPoblacion DestinatarioRegion  \
0  101  CAPITAL FEDERAL       CAPITAL FEDERAL    Capital Federal   
1  102          ALBERTI               ALBERTI       Buenos Aires   
2  103  ALMIRANTE BROWN               ADROGUE       Buenos Aires   
3  103  ALMIRANTE BROWN               BURZACO       Buenos Aires   
4  103  ALMIRANTE BROWN              CLAYPOLE       Buenos Aires   

   7C8R+R7, Isidro Casanova, Provincia de Buenos Aires  \
0                                               24.0     
1                                              200.0     
2                                               23.0     
3                                               24.0     
4                                               34.0     

   3RF8+5F Plaza Huincul, Neuquén, Argentina  \
0                               

Procesando Orígenes:   0%|          | 0/1113 [00:00<?, ?it/s]

Error en ruta: CAPITAL FEDERAL, Capital Federal -> 3RF8+5F Plaza Huincul, Neuquén, Argentina. Error: Error: REQUEST_DENIED
Error en ruta: CAPITAL FEDERAL, Capital Federal -> Av. del Petroleo Argentino 799-1199, Berisso, Provincia de Buenos Aires, Argentina. Error: Error: REQUEST_DENIED
Error en ruta: CAPITAL FEDERAL, Capital Federal -> RP88, Córdoba, Argentina. Error: Error: REQUEST_DENIED
Error en ruta: CAPITAL FEDERAL, Capital Federal -> 76HW+R7 San Lorenzo, Santa Fe, Argentina. Error: Error: REQUEST_DENIED
Error en ruta: CAPITAL FEDERAL, Capital Federal -> CX6V+33 Junín, Provincia de Buenos Aires, Argentina. Error: Error: REQUEST_DENIED
Error en ruta: CAPITAL FEDERAL, Capital Federal -> W2FM+Q2 Las Compuertas, Mendoza, Argentina. Error: Error: REQUEST_DENIED
Error en ruta: CAPITAL FEDERAL, Capital Federal -> 9FGG+3X Villa Mercedes, San Luis, Argentina. Error: Error: REQUEST_DENIED
Error en ruta: CAPITAL FEDERAL, Capital Federal -> Av. Monseñor Jesús Díaz 1125, T4109 Banda del Río Sa

KeyboardInterrupt: 

In [ ]:
# --- 1. Instalar biblioteca ---
!pip install googlemaps

# --- 2. Importar bibliotecas ---
import googlemaps
from google.colab import userdata
import json # Para imprimir errores bonitos

# --- 3. Leer la API Key del secreto 'maps' ---
try:
    API_KEY = userdata.get('maps')
    print("API Key leída del secreto 'maps'.")
    # Imprimimos solo los primeros 4 caracteres por seguridad
    print(f"Tu API Key empieza con: {API_KEY[:4]}...")
except Exception as e:
    print(f"--- ERROR AL LEER SECRETO ---")
    print(f"No se pudo encontrar el secreto 'maps'. Asegúrate de haberlo guardado.")
    print(f"Error: {e}")
    # Detenemos la ejecución si no hay clave
    raise SystemExit("Detenido por falta de API Key.")

# --- 4. Inicializar el cliente ---
try:
    gmaps = googlemaps.Client(key=API_KEY)
    print("Cliente de Google Maps inicializado.")
except Exception as e:
    print(f"--- ERROR AL INICIALIZAR CLIENTE ---")
    print(f"Error: {e}")
    raise SystemExit("Detenido por error de cliente.")


# --- 5. Direcciones de prueba Hardcodeadas ---
origen_test = "Obelisco, Buenos Aires, Argentina"
destino_test = "Catedral de La Plata, La Plata, Provincia de Buenos Aires, Argentina"

print(f"\nProbando ruta...")
print(f"  Origen: {origen_test}")
print(f"  Destino: {destino_test}")

# --- 6. La llamada a la API ---
try:
    # Esta es la función que usa la "Directions API"
    directions_result = gmaps.directions(origen_test,
                                         destino_test,
                                         mode="driving")

    # --- 7. Mostrar el resultado ---
    if directions_result:
        distance_meters = directions_result[0]['legs'][0]['distance']['value']
        dist_km = distance_meters / 1000

        print("\n" + "="*20)
        print("   ¡ÉXITO! LA API FUNCIONA")
        print(f"   Distancia calculada: {dist_km} km")
        print("="*20)
    else:
        print("\n" + "!"*20)
        print("   ERROR: ZERO_RESULTS")
        print("   La API funcionó, pero no se encontró una ruta.")
        print("!"*20)

except Exception as e:
    # --- 8. Mostrar el error ---
    print("\n" + "!"*20)
    print("   ¡ERROR! LA API HA FALLADO")
    print(f"   La API falló con el siguiente error:")
    print(f"   Tipo de Error: {type(e)}")
    print(f"   Mensaje Completo: {e}")
    print("!"*20)

API Key leída del secreto 'maps'.
Tu API Key empieza con: AIza...
Cliente de Google Maps inicializado.

Probando ruta...
  Origen: Obelisco, Buenos Aires, Argentina
  Destino: Catedral de La Plata, La Plata, Provincia de Buenos Aires, Argentina

   ¡ÉXITO! LA API FUNCIONA
   Distancia calculada: 57.608 km
